In [4]:
from elasticsearch import Elasticsearch
import pandas as pd

es = Elasticsearch([{'host': 'localhost', 'port': 9200,'scheme': 'http'}])      # Connexion à Elasticsearch

In [5]:
response = es.search(       # index du premier patient
    index="notes",
    body={
        "query": {"match_all": {}},
        "sort": {"date": {"order": "asc"}},
        "size": 1
    }
)
first_name = response["hits"]["hits"][0]["_source"]["patient_firstname"]  # Récupère le prénom
last_name = response["hits"]["hits"][0]["_source"]["patient_lastname"]  # Récupère le nom de famille
print(first_name, last_name)

/var/folders/_s/5mkyygcx5flgldzp7jhrqr5w0000gn/T/ipykernel_56492/3730916846.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(       # index du premier patient
/var/folders/_s/5mkyygcx5flgldzp7jhrqr5w0000gn/T/ipykernel_56492/3730916846.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(       # index du premier patient


IndexError: list index out of range

Requêtes

In [ ]:
result = es.search(index="notes", size=2)   #  Recupère les 2 premiers documents

for hit in result['hits']['hits']:      # Affiche les notes des résultats parcourus
    note = hit['_source']
    print("Patient Last Name:", note['patient_lastname'])
    print("Patient First Name:", note['patient_firstname'])
    print("Text:", note['Text'])
    print("Date:", note['date'])
    print("Patient Left:", note['patient_left'])
    print("Emotion:", note['Emotion'])
    print("Confidence:", note['confidence'])
    print("-------------------------")


Patient Last Name: Espinoza
Patient First Name: Yvonne
Text: i can t let go of that sad feeling that i want to be accepted here in this first home of mine
Date: 2022-08-15
Patient Left: True
Emotion: love
Confidence: 0.47055910806738893
-------------------------
Patient Last Name: Grant
Patient First Name: Sarah
Text: on a boat trip to denmark
Date: 2023-02-09
Patient Left: False
Emotion: happy
Confidence: 0.7427826342216294
-------------------------


/tmp/ipykernel_460618/272352446.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index="notes", size=2)   #  Recupère les 2 premiers documents


1 - Répartition des sentiments par patient

In [ ]:
def get_emotion_distribution(patient_lastname:str, patient_firstname:str) -> pd.DataFrame:
    """_summary_
    Prends le prenom et le nom d'un patient, et retourne le nombre de textes et son pourcentage pour chaque emotion
    """
    query = {           #requête
        "size": 0,
        "query": {
            "bool": {
                "must": [
                    {"term": {"patient_firstname": patient_firstname}},
                    {"term": {"patient_lastname": patient_lastname}}
                ]
            }
        },
        "aggs": {
            "sentiment_distribution": {
                "terms": {"field": "Emotion"}
            }
        }
    }
    result = es.search(index='notes', body=query)
    buckets = result["aggregations"]["sentiment_distribution"]["buckets"]
    df = pd.DataFrame(buckets, columns=["key", "doc_count"])    # crée le dataframe
    total_count = df["doc_count"].sum()
    df["Percent"] = round(((df["doc_count"] / total_count) * 100),2)    # Calcul des pourcentages
    return df

In [ ]:
get_emotion_distribution(last_name, first_name)

/tmp/ipykernel_460618/649475586.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index='notes', body=query)
/tmp/ipykernel_460618/649475586.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index='notes', body=query)


,key,doc_count,Percent
0,sadness,35,28.93
1,happy,34,28.10
2,fear,26,21.49
3,anger,17,14.05
4,love,6,4.96
5,surprise,3,2.48


2 - Matrice des sentiments contradictoires

In [ ]:
def emotion_count(es,emotion:str) -> int:
    query = {
        "size": 0,
        "query": {
            "terms": {
                "emotion": [emotion]
            }
        }
    }
    response = es.search(index="notes", body=query)
    return response["hits"]["total"]["value"]

In [ ]:
emotion_count(es, 'love')

/tmp/ipykernel_460618/527221375.py:10: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="notes", body=query)
/tmp/ipykernel_460618/527221375.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(index="notes", body=query)


1617

In [ ]:
def count_matching_entries(es, emotion, text):
    query = {
        "query": {
            "bool": {
                "must": [
                    {"match": {"emotion": emotion}},
                    {"match": {"Text": text}}
                ]
            }
        },
        "size": 0
    }

    response = es.search(index="notes", body=query)
    count = response["hits"]["total"]["value"]
    
    return count



In [ ]:
emotions = ['happy','sadness','anger','surprise','fear','love']

for e in emotions:
    for e2 in emotions:
        print(count_matching_entries(es, e, e2))

/tmp/ipykernel_460618/3550042845.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="notes", body=query)
/tmp/ipykernel_460618/3550042845.py:14: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(index="notes", body=query)


206
5
11
3
15
210
71
26
8
7
16
124
24
5
36
0
6
55
5
1
3
1
1
16
11
3
5
3
22
47
9
4
3
1
3
105


In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Connect to Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}]) # replace with your own host and port

# 1. Requête pour obtenir la répartition des sentiments pour un patient
def get_patient_sentiment_distribution(first_name, last_name):
    s = Search(using=es, index="your_index")         .query("match", first_name=first_name)         .query("match", last_name=last_name)         .aggs.bucket('sentiments', 'terms', field='sentiment')
    response = s.execute()
    df = pd.DataFrame(response.aggregations.sentiments.buckets)
    df.columns = ['sentiment', 'count']
    return df

# 2. Matrice des sentiments contradictoires
def get_sentiment_matrix():
    sentiments = ['happy', 'sadness', 'fear', 'anger', 'surprise', 'neutral']
    matrix = pd.DataFrame(index=sentiments, columns=sentiments)
    for sentiment in sentiments:
        for word in sentiments:
            s = Search(using=es, index="your_index")                 .query("match", sentiment=sentiment)                 .query("match", text=word)
            response = s.execute()
            matrix.loc[sentiment, word] = response.hits.total['value'] / response.hits.max_score
    sns.heatmap(matrix, annot=True, cmap='YlGnBu')
    plt.show()

# 3. Rechercher le nombre de textes correspondants aux étapes du deuil
def get_grief_stage_counts():
    stages = ['denial', 'anger', 'bargaining', 'depression', 'acceptance']
    counts = {}
    for stage in stages:
        # Full text search
        s = Search(using=es, index="your_index").query("match", text=stage)
        response = s.execute()
        counts[stage] = {
            'full_search': response.hits.total['value'],
            'fuzzy_search': 0
        }
        # Fuzzy search
        s = Search(using=es, index="your_index").query("fuzzy", text={"value": stage})
        response = s.execute()
        counts[stage]['fuzzy_search'] = response.hits.total['value']
    return pd.DataFrame(counts)

# 4. Rechercher les textes avec les conditions données
def get_filtered_texts():
    s = Search(using=es, index="your_index")         .query("match_phrase", text="good day")         .filter("term", in_consultation=True)         .should("match_phrase", text="to rest")         .exclude("range", confidence={"lt": 0.5})
    response = s.execute()
    df = pd.DataFrame((hit.to_dict() for hit in response), columns=['text', 'sentiment', 'confidence'])
    return df

ImportError: cannot import name 'Mapping' from 'elasticsearch.compat' (/home/apprenant/Projets/NLP/psycho_NLP/venv/lib/python3.10/site-packages/elasticsearch/compat.py)